<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Update BDD Growth

**Tags:** #googlesheets #gsheet #data #naas_drivers #operations #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook allows to send data to Google Sheets to a Google Sheets spreadsheet.

## Input

### Import libraries

In [ ]:
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date
import naas_data_product
try:
    import openai
except:
    !pip install openai --user
    import openai
import time

### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `file_reactions`: Name of the file with reactions to be retrieved.
- `file_comments`: Name of the file with comments to be retrieved.

**Outputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `append`: If False, data will be canceled and replaced.

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())
file_name = "linkedin_interactions"
openai_api_key = naas.secret.get("OPENAI_API_KEY")
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
ref_contact_name = "CONTACTS"
ref_company_name = "COMPANIES"

# Outputs
sheet_name = "GROWTH"
append = False

## Model

### Get BDD interactions

In [ ]:
df_interactions = pload(input_dir, file_name)    
print('🗂️ Interactions:', len(df_interactions))
df_interactions.head(1)

### Get data "GROWTH" from Google Sheets spreadsheet

In [ ]:
df_gsheet = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
print("Rows:", len(df_gsheet))
df_gsheet.head(1)

### Get data "REF_CONTACT" from Google Sheets spreadsheet

In [ ]:
ref_contact = gsheet.connect(spreadsheet_url).get(sheet_name=ref_contact_name)
print("Ref. contact:", len(ref_contact))
ref_contact.head(5)

### Get data "REF_COMPANY" from Google Sheets spreadsheet

In [ ]:
ref_company = gsheet.connect(spreadsheet_url).get(sheet_name=ref_company_name)
print("Ref. company:", len(ref_company))
ref_company.head(1)

### Create Growth dataframe

In [ ]:
def get_interactions_by_profile_and_scenario(
    df_init,
):
    # Init
    df = df_init.copy()
    df_interactions = pd.DataFrame()
    
    # Cleaning
    to_select = [
        "SCENARIO",
        "PROFILE_URL",
        "CONTENT_TITLE",
        "CONTENT_URL",
        "INTERACTION",
        "INTERACTION_CONTENT"
    ]
    df = df[to_select].sort_values(by="PROFILE_URL").reset_index(drop=True)
    df["INTERACTION_TEXT"] = ""
    df.loc[df["INTERACTION"] == "POST_REACTION", "INTERACTION_TEXT"] = df["INTERACTION_CONTENT"].str.capitalize() + " '" + df["CONTENT_TITLE"].str.strip() + "' (" + df["CONTENT_URL"] + ")"
    df.loc[df["INTERACTION"] == "POST_COMMENT", "INTERACTION_TEXT"] = "Comment '" + df["INTERACTION_CONTENT"].str.capitalize() + "' on '" + df["CONTENT_TITLE"].str.strip() + "' (" + df["CONTENT_URL"] + ")"

    # Create interactions by profile
    df_keys = df_init.copy()
    df_keys = df_keys[["SCENARIO", "PROFILE_URL"]].drop_duplicates()
    for row in df_keys.itertuples():
        index = row.Index
        scenario = row.SCENARIO
        profile_url = row.PROFILE_URL
        tmp_df = df.copy()
        tmp_df = tmp_df[(tmp_df["SCENARIO"] == scenario) & (tmp_df["PROFILE_URL"] == profile_url)].reset_index(drop=True)
        interests = ""
        for row in tmp_df.itertuples():
            interaction_text = row.INTERACTION_TEXT
            interests = f"{interests}{interaction_text}, "
        df_keys.loc[index, "INTERACTIONS"] = interests.strip()
    return df_keys

df_interaction_text = get_interactions_by_profile_and_scenario(df_interactions)
df_interaction_text

In [ ]:
df_last_interaction = df_interactions[["SCENARIO", "PROFILE_URL", "PUBLISHED_DATE"]].drop_duplicates().drop_duplicates(["SCENARIO", "PROFILE_URL"])
df_last_interaction

In [ ]:
df = df_interactions.copy()

# Groupby
to_group = [
    "ENTITY",
    "SCENARIO",
    "PLATFORM",
    "FULLNAME",
    "OCCUPATION",
    "PROFILE_URL",
]
to_agg = {
    "INTERACTION_SCORE": "sum"
}
df = df.groupby(to_group, as_index=False).agg(to_agg)

# Merge data
df = pd.merge(df, df_interaction_text, how="left")
df = pd.merge(df, df_last_interaction, how="left")
to_rename = {
    "PUBLISHED_DATE": "LAST_INTERACTION_DATE"
}
df = df.rename(columns=to_rename)
df["LAST_INTERACTION_DATE"] = pd.to_datetime(df["LAST_INTERACTION_DATE"].str[:-5]).dt.strftime("%a. %d %b.")
df = df.sort_values(by=["SCENARIO", "INTERACTION_SCORE"], ascending=[False, False]).reset_index(drop=True)
print("Rows:", len(df))
df.head(5)

## Output

### Send data to Google Sheets spreadsheet

In [ ]:
gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name, data=df, append=append)